# FIT9059 Assignment 2 - Spanning Trees and Clustering

Author: Lynn Miller

Student ID: 27610020

Date: 28/05/2016

Description: Code to implement Kruskal's algorithm to generate a minimum cost spanning tree on a partition of points. The code is then modified to generate clusters using single-link clustering

## Import all classes used

In [1]:
from math import *
import random


## 1. Function to Generate a Test Set of Random Points 

### 1.1 ADT Class for Points

The Point class defines a point in 2-dimensional Euclidean space. In addition to the constructor and string methods it contains these methods:
- distance_from_origin: returns the distance of the point from the origin
- distance: returns the distance from another point
- translate: moves a point to a new location
- equals: returns true if the "self" point represent the same location in the Euclidean space as the "other" point, otherwise it returns false
- isBefore: returns true if the "self" point is less than or equal to the "other" point in the requested dimension, otherwise is returns false
- getX and getY: return the x and y coordinates of the point, respectively 

In [2]:
class Point:

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return "(" + str(self.x) + ", " + str(self.y) + ")"
        
    def distance_from_origin(self):
        return sqrt(self.x * self.x + self.y * self.y)

    def distance(self, other):
        dx = self.x - other.x
        dy = self.y - other.y
        return sqrt(dx * dx + dy * dy)

    def translate(self, dx, dy):
        self.x += dx
        self.y += dy

    def equals(self, other):
        return self.x == other.x and self.y == other.y
    
    def isBefore(self, other, dim):
        if dim.lower() == "x":
            return self.x <= other.x
        elif dim.lower() == "y":
            return self.y <= other.y
        else:
            return None

    def getX(self):
        return self.x
    
    def getY(self):
        return self.y


### 1.2 Function to Generate Random Points

This function generates a set of <i>n</i> random points using the ADT class for points

In [3]:
def generateRandomPoints (numPoints):
    points = set()
    for n in range(numPoints):
        points.add(Point(random.randint(0,1000),random.randint(0,1000)))
    return points

## 2. Implement the Partition ADT

### 2.1 Define Node and Edge Classes

The Node class allows points to be organised into a tree structure. As well as the point attribute, it has a parent attribute, which allows a node to point to its parent node, a treeDepth that records the depth of the tree below the node and the incEdges attribute, that will be used later in the assignment to store the incident edges in the MCST. It has the following methods:
- the constructor method
- a method to represent the node as a string
- addSubTree: adds a sub-tree to this node and increments the tree depth if required 
- adjNodes: returns the set of adjacent nodes

The Edge class implements a graph edge between two nodes. It is not used until later in the assignment, but is defined here for convenience. It only has the constructor and string methods. The distance between the two points represented by the two nodes is stored as the weight of the edge.

In [4]:
class Node:
        
    def __init__ (self, point, depth=0):
        self.point = point
        self.parent = self
        self.treeDepth = depth
        self.incEdges = set()
        
    def __str__ (self):
        if self.parent == None:
            return "[" + str(self.point) + ",None]"
        else:    
            return "[" + str(self.point) + ',' + str(self.parent.point) + "]"
    
    def addSubTree (self, subTree):
        subTree.parent = self
        if subTree.treeDepth >= self.treeDepth:
            self.treeDepth = subTree.treeDepth + 1
            
    def adjNodes(self):
        nodeSet = set()
        for e in self.incEdges:
            if e.node1 == self:
                nodeSet.add(e.node2)
            else:
                nodeSet.add(e.node1)
        return nodeSet                

class Edge:
    
    def __init__ (self, node1, node2):
        self.node1 = node1
        self.node2 = node2
        self.weight = node1.point.distance(node2.point) 
        
    def __str__ (self):
        return str(self.node1.point) + " " + str(self.node2.point) + " " + str(self.weight)

### 2.2 The Partition ADT

The partition ADT contains the following methods. All the nodes in the partition are stored in a directory. The keys of the directory are the x and y coordinates of the point in each node. The sets are identified by the tree structures linking the nodes.

The partition ADT has these methods:
- The constructor method: takes an optional parameter of a list of points to add to the partition. 
- newSet: creates a new set in the partition. It creates a node for the point and adds that node to the dictionary of vertices.
- find: finds the root node of a given node by following the parent nodes until a node with no parent is found
- union: unions two sets by finding the roots of the two nodes. If the roots are the same, the nodes are in the same set and no further action is required. If the roots are different, the nodes are in separate sets, the sets are unioned by setting the parent of the tree with the shortest depth to the root of the tree with the greatest depth. The tree depth is adjusted if required. The method returns a boolean value indicating whether or not the union was needed.
- getNode: used for testing purposes and returns the node containing the specified point
- There is also a method to return a string representation of the partition

In [5]:
class Partition:
    
    def __init__ (self, pointSet=set()):
        self.vertices = {}
        self.partitions = 0
        for point in pointSet:
            self.newSet(point)
        
    def newSet (self, point):
        self.vertices[(point.getX(),point.getY())] = Node(point)
        self.partitions = self.partitions + 1
 
    def getNode (self, point):
        return self.vertices[(point.getX(),point.getY())]
        
    def union (self, node1, node2):
        root1 = self.find(node1)
        root2 = self.find(node2)
        if root1 == root2:
            return False
        if root1.treeDepth < root2.treeDepth:
            root2.addSubTree(root1)
        else:
            root1.addSubTree(root2)
        self.partitions = self.partitions-1
        return True
        
    def find (self, node):
        if node.parent == node:
            return node
        else:
            return self.find(node.parent)
        
    def __str__ (self):
        output = "[" + str(self.partitions) + "]:{"
        for i, v in self.vertices.items():
            output = output + str(v)
        output = output + "}"
        return output
    

### 2.3 Test the Partition ADT Class
Test the Partition ADT class by creating some random points, using these points to create the partition and running the methods of the ADT.

In [6]:
# Generate the points
points = generateRandomPoints(20)

# See the points generated
for point in points:
    print(point)
    
# Create the partition    
part = Partition(points)

# See the points in the partition
print("")
print("Partition before UnionFind testing:")
print(part)

# Test the union operation (also tests the find)
pointIter = iter(points)
node0 = part.getNode(next(pointIter))
node1 = part.getNode(next(pointIter))
part.union(node0, node1)

# See how the partition has changed
print("")
print("Partition after UnionFind testing:")
print(part)

# Print the point and parent details for the two nodes
print("")
print("Point 0 : " + str(node0))
print("Point 1 : " + str(node1))

(516, 710)
(321, 31)
(346, 401)
(892, 845)
(394, 377)
(226, 9)
(81, 575)
(138, 517)
(519, 418)
(244, 174)
(496, 486)
(922, 632)
(461, 476)
(472, 362)
(824, 57)
(896, 738)
(122, 48)
(450, 707)
(5, 403)
(826, 116)

Partition before UnionFind testing:
[20]:{[(496, 486),(496, 486)][(138, 517),(138, 517)][(516, 710),(516, 710)][(226, 9),(226, 9)][(922, 632),(922, 632)][(824, 57),(824, 57)][(896, 738),(896, 738)][(394, 377),(394, 377)][(81, 575),(81, 575)][(472, 362),(472, 362)][(461, 476),(461, 476)][(450, 707),(450, 707)][(346, 401),(346, 401)][(826, 116),(826, 116)][(892, 845),(892, 845)][(122, 48),(122, 48)][(5, 403),(5, 403)][(244, 174),(244, 174)][(519, 418),(519, 418)][(321, 31),(321, 31)]}

Partition after UnionFind testing:
[19]:{[(496, 486),(496, 486)][(138, 517),(138, 517)][(516, 710),(516, 710)][(226, 9),(226, 9)][(922, 632),(922, 632)][(824, 57),(824, 57)][(896, 738),(896, 738)][(394, 377),(394, 377)][(81, 575),(81, 575)][(472, 362),(472, 362)][(461, 476),(461, 476)][(450, 707),

## 3 Implement the Clustering Procedure

### 3.1 Extend the Partition ADT to support the clustering procedure

The following variables and methods are added to the partition ADT to support clustering
- candEdges: A priority queue that contains all possible edges that have not been used to cluster the nodes (candidate edges)
- numCandidates: The number of edges in candEdges
- addCandEdges: This method is used when adding a node to the partition. It adds an edge from the new node to every existing node to candEdges as candidate edges, maintaining the priority queue as the edges are added.
- getEdge: Returns the first edge from candEdges, removing it as a candidate edge while maintaining the priority queue.
- cluster: Looks at each candidate edge and adds it to the tree if it joins two subtrees. The edge is discarded if it does not join two subtrees (i.e. it is an edge betweeb two nodes already in the same tree). Halts when the partition consists of a single set.

In [7]:
class Partition:
    
    def __init__ (self, pointSet=set()):
        self.vertices = {}
        self.candEdges = []
        self.partitions = 0
        self.numCandidates = 0
        for point in pointSet:
            self.newSet(point)
        
    def newSet (self, point):
        newNode = Node(point)
        self.addCandEdges(newNode)
        self.vertices[(point.getX(),point.getY())] = newNode
        self.partitions = self.partitions + 1
 
    def getNode (self, point):
        return self.vertices[(point.getX(),point.getY())]
      
    def addCandEdges (self, node):
        for i, v in self.vertices.items():
            self.candEdges.append(Edge(node, v))
            self.numCandidates = self.numCandidates + 1
            e = self.numCandidates - 1
            done = False
            while e > 0 and not done:
                p = floor((e-1)/2)
                if self.candEdges[e].weight < self.candEdges[p].weight:
                    self.candEdges[e], self.candEdges[p] = self.candEdges[p], self.candEdges[e]
                    e = p
                else:
                    done = True
                    
    def getEdge (self):
        if self.numCandidates == 0:
            return None
        nextEdge = self.candEdges[0]
        self.numCandidates = self.numCandidates - 1
        self.candEdges[0] = self.candEdges[self.numCandidates]
        self.candEdges.pop(self.numCandidates)
        p = 0
        while p * 2 + 1 < self.numCandidates:
            e1 = p * 2 + 1
            e2 = e1 + 1
            if e2 >= self.numCandidates:
                self.candEdges[e1], self.candEdges[p] = self.candEdges[p], self.candEdges[e1]
                p = e1
            elif self.candEdges[e1].weight < self.candEdges[e2].weight:
                self.candEdges[e1], self.candEdges[p] = self.candEdges[p], self.candEdges[e1]
                p = e1
            else:    
                self.candEdges[e2], self.candEdges[p] = self.candEdges[p], self.candEdges[e2]
                p = e2
        return nextEdge
        
    def union (self, node1, node2):
        root1 = self.find(node1)
        root2 = self.find(node2)
        if root1 == root2:
            return False
        if root1.treeDepth < root2.treeDepth:
            root2.addSubTree(root1)
        else:
            root1.addSubTree(root2)
        self.partitions = self.partitions-1
        return True

    def find (self, node):
        if node.parent == node:
            return node
        else:
            return self.find(node.parent)
        
    def cluster (self):
        while self.partitions > 1:
            nextEdge = self.getEdge()
            if self.union(nextEdge.node1, nextEdge.node2):
                print("Joined points " + str(nextEdge.node1.point) + " and " +
                      str(nextEdge.node2.point) + " : " + str(nextEdge.weight))
                nextEdge.node1.incEdges.add(nextEdge)
                nextEdge.node2.incEdges.add(nextEdge)
           
    def __str__ (self):
        output = "[" + str(self.partitions) + "]:{"
        for i, v in self.vertices.items():
            output = output + str(v)
        output = output + "}"
        return output
    
    def printCandEdges (self):
        for e in iter(self.candEdges):
            print(str(e))
    

### 3.2 Test the Cluster Method

Use the previously generated set of points to create a partition and then cluster them to generate the MCST 

In [8]:
part = Partition(points)
print(part)
print("")

# Create the MCST
part.cluster()

# Print the partition
print("")
print(part)

[20]:{[(496, 486),(496, 486)][(138, 517),(138, 517)][(516, 710),(516, 710)][(226, 9),(226, 9)][(922, 632),(922, 632)][(824, 57),(824, 57)][(896, 738),(896, 738)][(394, 377),(394, 377)][(81, 575),(81, 575)][(472, 362),(472, 362)][(461, 476),(461, 476)][(450, 707),(450, 707)][(346, 401),(346, 401)][(826, 116),(826, 116)][(892, 845),(892, 845)][(122, 48),(122, 48)][(5, 403),(5, 403)][(244, 174),(244, 174)][(519, 418),(519, 418)][(321, 31),(321, 31)]}

Joined points (461, 476) and (496, 486) : 36.40054944640259
Joined points (394, 377) and (346, 401) : 53.665631459994955
Joined points (826, 116) and (824, 57) : 59.033888572581766
Joined points (450, 707) and (516, 710) : 66.06814663663572
Joined points (496, 486) and (519, 418) : 71.78439941937245
Joined points (472, 362) and (519, 418) : 73.10950690573696
Joined points (472, 362) and (394, 377) : 79.42921376924235
Joined points (138, 517) and (81, 575) : 81.32035415564789
Joined points (226, 9) and (321, 31) : 97.51410154434076
Joined poi

### 3.3 A Further Check using BFS

As a further check, I've defined a BFS function that can be used to check the MCST contains all the nodes. BFS code based on Edd Mann's code (http://eddmann.com/posts/depth-first-search-and-breadth-first-search-in-python/) and modified to work with the partition ADT and multiple clusters.

In [9]:
def bfs(part):
    clusters = []
    queue = []
    allVisited = set()
    for v in part.vertices:
        if not (part.vertices[v] in allVisited):
            queue.append(part.vertices[v])
            visited = set()
            while queue:
                vertex = queue.pop(0)
                if vertex not in allVisited:
                    visited.add(vertex)
                    allVisited.add(vertex)
                    queue.extend(vertex.adjNodes() - visited)
            clusters.append(visited)
    return clusters

clusters = []
clusters = bfs(part)
print("Clusters in Partition")
print("=====================")
for i in iter(clusters):
    print("Next Cluster length: ",len(i))
    for j in i:
        print(j)
    print("==========")    


Clusters in Partition
Next Cluster length:  20
[(346, 401),(394, 377)]
[(138, 517),(461, 476)]
[(122, 48),(226, 9)]
[(5, 403),(138, 517)]
[(516, 710),(450, 707)]
[(496, 486),(461, 476)]
[(472, 362),(461, 476)]
[(450, 707),(461, 476)]
[(244, 174),(226, 9)]
[(81, 575),(138, 517)]
[(922, 632),(896, 738)]
[(892, 845),(896, 738)]
[(394, 377),(461, 476)]
[(226, 9),(461, 476)]
[(321, 31),(226, 9)]
[(896, 738),(461, 476)]
[(519, 418),(461, 476)]
[(461, 476),(461, 476)]
[(826, 116),(461, 476)]
[(824, 57),(826, 116)]


## 4 Add in Path Compression

### 4.1 Change the Node Class

The Node class is changed to maintain a node count instead of the tree depth - after sufficient find operations the tree depth is always 1.  

In [10]:
class Node:
        
    def __init__ (self, point, nodeCount = 1):
        self.point = point
        self.parent = self
        self.nodeCount = nodeCount
        self.incEdges = set()
        
    def __str__ (self):
        return "[" + str(self.point) + ',' + str(self.parent.point) + "]"
    
    def addSubTree (self, subTree):
        subTree.parent = self
        self.nodeCount = self.nodeCount + subTree.nodeCount;

    def adjNodes(self):
        nodeSet = set()
        for e in self.incEdges:
            if e.node1 == self:
                nodeSet.add(e.node2)
            else:
                nodeSet.add(e.node1)
        return nodeSet
    

### 4.2 Add Path Compression to the Partition ADT

Change the find method of the partition ADT to update the parents of all nodes traversed when finding the root node to be the root node. The root node is now defined as a node that is its own parent.

In [11]:
class Partition:
    
    def __init__ (self, pointSet=set()):
        self.vertices = {}
        self.candEdges = []
        self.partitions = 0
        self.numCandidates = 0
        for point in pointSet:
            self.newSet(point)
        
    def newSet (self, point):
        newNode = Node(point)
        self.addCandEdges(newNode)
        self.vertices[(point.getX(),point.getY())] = newNode
        self.partitions = self.partitions + 1
 
    def getNode (self, point):
        return self.vertices[(point.getX(),point.getY())]
      
    def addCandEdges (self, node):
        for i, v in self.vertices.items():
            self.candEdges.append(Edge(node, v))
            self.numCandidates = self.numCandidates + 1
            e = self.numCandidates - 1
            done = False
            while e > 0 and not done:
                p = floor((e-1)/2)
                if self.candEdges[e].weight < self.candEdges[p].weight:
                    self.candEdges[e], self.candEdges[p] = self.candEdges[p], self.candEdges[e]
                    e = p
                else:
                    done = True
                    
    def getEdge (self):
        if self.numCandidates == 0:
            return None
        nextEdge = self.candEdges[0]
        self.numCandidates = self.numCandidates - 1
        self.candEdges[0] = self.candEdges[self.numCandidates]
        self.candEdges.pop(self.numCandidates)
        p = 0
        while p * 2 + 1 < self.numCandidates:
            e1 = p * 2 + 1
            e2 = e1 + 1
            if e2 >= self.numCandidates:
                self.candEdges[e1], self.candEdges[p] = self.candEdges[p], self.candEdges[e1]
                p = e1
            elif self.candEdges[e1].weight < self.candEdges[e2].weight:
                self.candEdges[e1], self.candEdges[p] = self.candEdges[p], self.candEdges[e1]
                p = e1
            else:    
                self.candEdges[e2], self.candEdges[p] = self.candEdges[p], self.candEdges[e2]
                p = e2
        return nextEdge
        
    def union (self, node1, node2):
        root1 = self.find(node1)
        root2 = self.find(node2)
        if root1 == root2:
            return False
        if root1.nodeCount < root2.nodeCount:
            root2.addSubTree(root1)
        else:
            root1.addSubTree(root2)
        self.partitions = self.partitions-1
        return True

    def find (self, node):
        if node.parent != node.parent.parent:
            node.parent = self.find(node.parent)
        return node.parent
    
    def cluster (self):
        while self.partitions > 1:
            nextEdge = self.getEdge()
            if self.union(nextEdge.node1, nextEdge.node2):
                nextEdge.node1.incEdges.add(nextEdge)
                nextEdge.node2.incEdges.add(nextEdge)
           
    def __str__ (self):
        output = "[" + str(self.partitions) + "]:{"
        for i, v in self.vertices.items():
            output = output + str(v)
        output = output + "}"
        return output
    
    def printCandEdges (self):
        for e in iter(self.candEdges):
            print(str(e))
    

In [12]:
# Generate the partition
part = Partition(points)

# Print the partition prior to clustering
print(part)
print("")

# Cluster the partition
part.cluster()

# print the cluster
clusters = []
clusters = bfs(part)
print("Clusters in Partition")
print("=====================")
for i in iter(clusters):
    print("Next Cluster length: ",len(i))
    for j in i:
        print(j)
    print("==========")    


[20]:{[(496, 486),(496, 486)][(138, 517),(138, 517)][(516, 710),(516, 710)][(226, 9),(226, 9)][(922, 632),(922, 632)][(824, 57),(824, 57)][(896, 738),(896, 738)][(394, 377),(394, 377)][(81, 575),(81, 575)][(472, 362),(472, 362)][(461, 476),(461, 476)][(450, 707),(450, 707)][(346, 401),(346, 401)][(826, 116),(826, 116)][(892, 845),(892, 845)][(122, 48),(122, 48)][(5, 403),(5, 403)][(244, 174),(244, 174)][(519, 418),(519, 418)][(321, 31),(321, 31)]}

Clusters in Partition
Next Cluster length:  20
[(922, 632),(461, 476)]
[(496, 486),(461, 476)]
[(346, 401),(461, 476)]
[(892, 845),(461, 476)]
[(394, 377),(461, 476)]
[(122, 48),(461, 476)]
[(516, 710),(461, 476)]
[(138, 517),(461, 476)]
[(5, 403),(461, 476)]
[(81, 575),(461, 476)]
[(461, 476),(461, 476)]
[(826, 116),(461, 476)]
[(472, 362),(461, 476)]
[(450, 707),(461, 476)]
[(244, 174),(461, 476)]
[(519, 418),(461, 476)]
[(321, 31),(461, 476)]
[(226, 9),(461, 476)]
[(896, 738),(461, 476)]
[(824, 57),(826, 116)]


## 5 Implement K-Clustering

Change the Partition ADT to create k clusters, where k is a parameter passed to the clustering method.

The cluster method is changed to stop when there are k clusters, instead of running until there is only one cluster.

In [13]:
class Partition:
    
    def __init__ (self, pointSet=set()):
        self.vertices = {}
        self.candEdges = []
        self.partitions = 0
        self.numCandidates = 0
        for point in pointSet:
            self.newSet(point)
        
    def newSet (self, point):
        newNode = Node(point)
        self.addCandEdges(newNode)
        self.vertices[(point.getX(),point.getY())] = newNode
        self.partitions = self.partitions + 1
 
    def getNode (self, point):
        return self.vertices[(point.getX(),point.getY())]
      
    def addCandEdges (self, node):
        for i, v in self.vertices.items():
            self.candEdges.append(Edge(node, v))
            self.numCandidates = self.numCandidates + 1
            e = self.numCandidates - 1
            done = False
            while e > 0 and not done:
                p = floor((e-1)/2)
                if self.candEdges[e].weight < self.candEdges[p].weight:
                    self.candEdges[e], self.candEdges[p] = self.candEdges[p], self.candEdges[e]
                    e = p
                else:
                    done = True
                    
    def getEdge (self):
        if self.numCandidates == 0:
            return None
        nextEdge = self.candEdges[0]
        self.numCandidates = self.numCandidates - 1
        self.candEdges[0] = self.candEdges[self.numCandidates]
        self.candEdges.pop(self.numCandidates)
        p = 0
        while p * 2 + 1 < self.numCandidates:
            e1 = p * 2 + 1
            e2 = e1 + 1
            if e2 >= self.numCandidates:
                self.candEdges[e1], self.candEdges[p] = self.candEdges[p], self.candEdges[e1]
                p = e1
            elif self.candEdges[e1].weight < self.candEdges[e2].weight:
                self.candEdges[e1], self.candEdges[p] = self.candEdges[p], self.candEdges[e1]
                p = e1
            else:    
                self.candEdges[e2], self.candEdges[p] = self.candEdges[p], self.candEdges[e2]
                p = e2
        return nextEdge
        
    def union (self, node1, node2):
        root1 = self.find(node1)
        root2 = self.find(node2)
        if root1 == root2:
            return False
        if root1.nodeCount < root2.nodeCount:
            root2.addSubTree(root1)
        else:
            root1.addSubTree(root2)
        self.partitions = self.partitions-1
        return True

    def find (self, node):
        if node.parent != node.parent.parent:
            node.parent = self.find(node.parent)
        return node.parent
    
    def cluster (self, k=1):
        while self.partitions > k:
            nextEdge = self.getEdge()
            if self.union(nextEdge.node1, nextEdge.node2):
                nextEdge.node1.incEdges.add(nextEdge)
                nextEdge.node2.incEdges.add(nextEdge)
           
    def __str__ (self):
        output = "[" + str(self.partitions) + "]:{"
        for i, v in self.vertices.items():
            output = output + str(v)
        output = output + "}"
        return output
    
    def printCandEdges (self):
        for e in iter(self.candEdges):
            print(str(e))
    

In [14]:
part = Partition(points)

# Cluster the partition
part.cluster(4)

# Print the clusters
clusters = []
clusters = bfs(part)
print("Clusters in Partition")
print("=====================")
for i in iter(clusters):
    print("Next Cluster length: ",len(i))
    for j in i:
        print(j)
    print("==========")    


Clusters in Partition
Next Cluster length:  11
[(472, 362),(461, 476)]
[(5, 403),(138, 517)]
[(346, 401),(461, 476)]
[(461, 476),(461, 476)]
[(138, 517),(461, 476)]
[(519, 418),(461, 476)]
[(450, 707),(461, 476)]
[(81, 575),(138, 517)]
[(496, 486),(461, 476)]
[(516, 710),(450, 707)]
[(394, 377),(461, 476)]
Next Cluster length:  4
[(244, 174),(226, 9)]
[(321, 31),(226, 9)]
[(226, 9),(226, 9)]
[(122, 48),(226, 9)]
Next Cluster length:  3
[(922, 632),(896, 738)]
[(892, 845),(896, 738)]
[(896, 738),(896, 738)]
Next Cluster length:  2
[(826, 116),(826, 116)]
[(824, 57),(826, 116)]


## 6 Implement a Method to let the User Query if two points are in the same Cluster

### 6.1 Update the Partition ADT

Add the sameCluster method, which takes two points as parameters. It returns "TRUE" if the points are in the same cluster and "FALSE" otherwise

In [15]:
class Partition:
    
    def __init__ (self, pointSet=set()):
        self.vertices = {}
        self.candEdges = []
        self.partitions = 0
        self.numCandidates = 0
        for point in pointSet:
            self.newSet(point)
        
    def newSet (self, point):
        newNode = Node(point)
        self.addCandEdges(newNode)
        self.vertices[(point.getX(),point.getY())] = newNode
        self.partitions = self.partitions + 1
 
    def getNode (self, point):
        return self.vertices[(point.getX(),point.getY())]
      
    def addCandEdges (self, node):
        for i, v in self.vertices.items():
            self.candEdges.append(Edge(node, v))
            self.numCandidates = self.numCandidates + 1
            e = self.numCandidates - 1
            done = False
            while e > 0 and not done:
                p = floor((e-1)/2)
                if self.candEdges[e].weight < self.candEdges[p].weight:
                    self.candEdges[e], self.candEdges[p] = self.candEdges[p], self.candEdges[e]
                    e = p
                else:
                    done = True
                    
    def getEdge (self):
        if self.numCandidates == 0:
            return None
        nextEdge = self.candEdges[0]
        self.numCandidates = self.numCandidates - 1
        self.candEdges[0] = self.candEdges[self.numCandidates]
        self.candEdges.pop(self.numCandidates)
        p = 0
        while p * 2 + 1 < self.numCandidates:
            e1 = p * 2 + 1
            e2 = e1 + 1
            if e2 >= self.numCandidates:
                self.candEdges[e1], self.candEdges[p] = self.candEdges[p], self.candEdges[e1]
                p = e1
            elif self.candEdges[e1].weight < self.candEdges[e2].weight:
                self.candEdges[e1], self.candEdges[p] = self.candEdges[p], self.candEdges[e1]
                p = e1
            else:    
                self.candEdges[e2], self.candEdges[p] = self.candEdges[p], self.candEdges[e2]
                p = e2
        return nextEdge
        
    def union (self, node1, node2):
        root1 = self.find(node1)
        root2 = self.find(node2)
        if root1 == root2:
            return False
        if root1.nodeCount < root2.nodeCount:
            root2.addSubTree(root1)
        else:
            root1.addSubTree(root2)
        self.partitions = self.partitions-1
        return True

    def find (self, node):
        if node.parent != node.parent.parent:
            node.parent = self.find(node.parent)
        return node.parent
    
    def cluster (self, k=1):
        while self.partitions > k:
            nextEdge = self.getEdge()
            if self.union(nextEdge.node1, nextEdge.node2):
                nextEdge.node1.incEdges.add(nextEdge)
                nextEdge.node2.incEdges.add(nextEdge)

    def sameCluster (self, p1, p2):
        node1 = self.vertices[(p1.getX(),p1.getY())]
        node2 = self.vertices[(p2.getX(),p2.getY())]
        root1 = self.find(node1)
        root2 = self.find(node2)
        if root1 == root2:
            return True
        else:
            return False  
        
    def __str__ (self):
        output = "[" + str(self.partitions) + "]:{"
        for i, v in self.vertices.items():
            output = output + str(v)
        output = output + "}"
        return output
    
    def printCandEdges (self):
        for e in iter(self.candEdges):
            print(str(e))
    

### 6.2 Test the sameCluster Method

Generate a partition and run a test of the sameCluster method

In [16]:
part = Partition(points)
part.cluster(4)

clusters = []
clusters = bfs(part)
print("Clusters in Partition")
print("=====================")
for i in iter(clusters):
    print("Next Cluster length: ",len(i))
    for j in i:
        print(j)
    print("==========")    

pointIter = iter(points)
p1 = next(pointIter)
p2 = next(pointIter)
print("")
if part.sameCluster(p1,p2):
    print ("Points "+str(p1)+" and "+str(p2)+" are in the same cluster")
else:
    print ("Points "+str(p1)+" and "+str(p2)+" are in different clusters")

Clusters in Partition
Next Cluster length:  11
[(394, 377),(461, 476)]
[(516, 710),(450, 707)]
[(472, 362),(461, 476)]
[(496, 486),(461, 476)]
[(81, 575),(138, 517)]
[(450, 707),(461, 476)]
[(346, 401),(461, 476)]
[(5, 403),(138, 517)]
[(461, 476),(461, 476)]
[(519, 418),(461, 476)]
[(138, 517),(461, 476)]
Next Cluster length:  4
[(321, 31),(226, 9)]
[(122, 48),(226, 9)]
[(244, 174),(226, 9)]
[(226, 9),(226, 9)]
Next Cluster length:  3
[(922, 632),(896, 738)]
[(892, 845),(896, 738)]
[(896, 738),(896, 738)]
Next Cluster length:  2
[(824, 57),(826, 116)]
[(826, 116),(826, 116)]

Points (516, 710) and (321, 31) are in different clusters


## 7 Function to Calculate the Dunn Index

### 7.1 Minor Update to the Partition ADT

Some modifications have been made to the partition ADT to support calculating the Dunn index:
- lastEdgeWeight: a variable called lastEdgeWeight has been added to the ADT and the cluster function sets this to the weight of the last edge added when generating the clusters.
- maxEdgeWeight: a method to return the maximum edge weight in the graph, which is the lastEdgeWeight value


In [17]:
class Partition:
    
    def __init__ (self, pointSet=set()):
        self.vertices = {}
        self.candEdges = []
        self.partitions = 0
        self.numCandidates = 0
        self.lastEdge = 0
        for point in pointSet:
            self.newSet(point)
        
    def newSet (self, point):
        newNode = Node(point)
        self.addCandEdges(newNode)
        self.vertices[(point.getX(),point.getY())] = newNode
        self.partitions = self.partitions + 1
 
    def getNode (self, point):
        return self.vertices[(point.getX(),point.getY())]
      
    def addCandEdges (self, node):
        for i, v in self.vertices.items():
            self.candEdges.append(Edge(node, v))
            self.numCandidates = self.numCandidates + 1
            e = self.numCandidates - 1
            done = False
            while e > 0 and not done:
                p = floor((e-1)/2)
                if self.candEdges[e].weight < self.candEdges[p].weight:
                    self.candEdges[e], self.candEdges[p] = self.candEdges[p], self.candEdges[e]
                    e = p
                else:
                    done = True
                    
    def getEdge (self):
        if self.numCandidates == 0:
            return None
        nextEdge = self.candEdges[0]
        self.numCandidates = self.numCandidates - 1
        self.candEdges[0] = self.candEdges[self.numCandidates]
        self.candEdges.pop(self.numCandidates)
        p = 0
        while p * 2 + 1 < self.numCandidates:
            e1 = p * 2 + 1
            e2 = e1 + 1
            if e2 >= self.numCandidates:
                self.candEdges[e1], self.candEdges[p] = self.candEdges[p], self.candEdges[e1]
                p = e1
            elif self.candEdges[e1].weight < self.candEdges[e2].weight:
                self.candEdges[e1], self.candEdges[p] = self.candEdges[p], self.candEdges[e1]
                p = e1
            else:    
                self.candEdges[e2], self.candEdges[p] = self.candEdges[p], self.candEdges[e2]
                p = e2
        return nextEdge
        
    def union (self, node1, node2):
        root1 = self.find(node1)
        root2 = self.find(node2)
        if root1 == root2:
            return False
        if root1.nodeCount < root2.nodeCount:
            root2.addSubTree(root1)
        else:
            root1.addSubTree(root2)
        self.partitions = self.partitions-1
        return True

    def find (self, node):
        if node.parent != node.parent.parent:
            node.parent = self.find(node.parent)
        return node.parent
    
    def cluster (self, k=1):
        while self.partitions > k:
            nextEdge = self.getEdge()
            if self.union(nextEdge.node1, nextEdge.node2):
                nextEdge.node1.incEdges.add(nextEdge)
                nextEdge.node2.incEdges.add(nextEdge)
        self.lastEdge = nextEdge

    def sameCluster (self, p1, p2):
        node1 = self.vertices[(p1.getX(),p1.getY())]
        node2 = self.vertices[(p2.getX(),p2.getY())]
        root1 = self.find(node1)
        root2 = self.find(node2)
        if root1 == root2:
            return True
        else:
            return False 
        
    def maxWeightEdge(self):
        return self.lastEdge
    
    def __str__ (self):
        output = "[" + str(self.partitions) + "]:{"
        for i, v in self.vertices.items():
            output = output + str(v)
        output = output + "}"
        return output
    
    def printCandEdges (self):
        for e in iter(self.candEdges):
            print(str(e))
    

### 7.2 The dunnIndex Function

The dunnIndex function calculates the Dunn index by considering the candidate edges that were not considered when building the clusters. The index is calculated as (minimum inter-cluster distance)/(maximum intra-cluster distance) 

- The minimum inter-cluster distance is the shortest distance between two points in different clusters.
- The maximum intra-cluster distance is the longest distance between two points in the same cluster. This is initialised to the maximum edge weight in the graph, which is needed in case all intra-cluster edges were considered while building the clusters (this is unlikely but possible e.g. if all clusters contain only two points).

In [18]:
def dunnIndex (part):
    lastEdge = part.maxWeightEdge()
    intraClusterEdge = lastEdge
    interCluster = None 
    while part.numCandidates > 0:
        nextEdge = part.getEdge()
        if part.sameCluster (nextEdge.node1.point, nextEdge.node2.point):
            intraClusterEdge = nextEdge
        else:
            if interCluster == None:
                interCluster = nextEdge.weight
                print("Minimum inter-cluster weight: " + str(interCluster) + 
                      " (points " + str(nextEdge.node1.point) + "," + str(nextEdge.node2.point) + ")")
    print("Maximum intra-cluster weight: " + str(intraClusterEdge.weight) + 
          " (points " + str(intraClusterEdge.node1.point) + "," + str(intraClusterEdge.node2.point) + ")")
    return interCluster/intraClusterEdge.weight            


In [19]:
part = Partition(points)
part.cluster(4)

clusters = []
clusters = bfs(part)
print("Clusters in Partition")
print("=====================")
for i in iter(clusters):
    print("Next Cluster length: ",len(i))
    for j in i:
        print(j)
    print("==========")    

print("")    
print("Dunn Index = ",str(dunnIndex(part)))

Clusters in Partition
Next Cluster length:  11
[(519, 418),(461, 476)]
[(81, 575),(138, 517)]
[(516, 710),(450, 707)]
[(394, 377),(461, 476)]
[(472, 362),(461, 476)]
[(461, 476),(461, 476)]
[(450, 707),(461, 476)]
[(5, 403),(138, 517)]
[(138, 517),(461, 476)]
[(496, 486),(461, 476)]
[(346, 401),(461, 476)]
Next Cluster length:  4
[(321, 31),(226, 9)]
[(244, 174),(226, 9)]
[(226, 9),(226, 9)]
[(122, 48),(226, 9)]
Next Cluster length:  3
[(922, 632),(896, 738)]
[(892, 845),(896, 738)]
[(896, 738),(896, 738)]
Next Cluster length:  2
[(824, 57),(826, 116)]
[(826, 116),(826, 116)]

Minimum inter-cluster weight: 248.86341635523692 (points (244, 174),(346, 401))
Maximum intra-cluster weight: 468.7963310436634 (points (5, 403),(472, 362))
Dunn Index =  0.5308561519694529


## 8 Compare k-Cluster Forest to Minimum Cost Spanning Tree

The set of edges in the MCST but not in the forest of k-clustering is the set of the last k-1 edges added to the tree when constructing the MCST. As the edges are added in order of cost, these are the k-1 most expensive edges in the MCST. 
